In [ ]:
import pandas as pd
from sklearn import *
import os
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import KNNImputer
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rohanrao/air-quality-data-in-india")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12


In [ ]:
# prompt: use this "/root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12"

import os

# Assuming the path is correctly set in the previous code execution.
# If not, replace with the actual path.
path = "/root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12"

# List all files within the specified directory
files_in_path = os.listdir(path)
print("Files in the specified directory:")
for file in files_in_path:
  print(file)

import glob
csv_files = glob.glob(os.path.join(path, "*.csv"))

for csv_file in csv_files:
  try:
    df = pd.read_csv(csv_file)
    print(f"\nData from {csv_file}:")
    print(df.head())
  except pd.errors.ParserError:
    print(f"Error parsing file {csv_file}")

Files in the specified directory:
city_hour.csv
station_hour.csv
station_day.csv
city_day.csv
stations.csv

Data from /root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12/city_hour.csv:
        City             Datetime  PM2.5  PM10    NO    NO2    NOx  NH3    CO  \
0  Ahmedabad  2015-01-01 01:00:00    NaN   NaN  1.00  40.01  36.37  NaN  1.00   
1  Ahmedabad  2015-01-01 02:00:00    NaN   NaN  0.02  27.75  19.73  NaN  0.02   
2  Ahmedabad  2015-01-01 03:00:00    NaN   NaN  0.08  19.32  11.08  NaN  0.08   
3  Ahmedabad  2015-01-01 04:00:00    NaN   NaN  0.30  16.45   9.20  NaN  0.30   
4  Ahmedabad  2015-01-01 05:00:00    NaN   NaN  0.12  14.90   7.85  NaN  0.12   

      SO2      O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  122.07     NaN      0.0      0.0     0.0  NaN        NaN  
1   85.90     NaN      0.0      0.0     0.0  NaN        NaN  
2   52.83     NaN      0.0      0.0     0.0  NaN        NaN  
3   39.53  153.58      0.0      0.0     0.0  NaN     

<ipython-input-113-d2867be10a16>:20: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)



Data from /root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12/station_hour.csv:
  StationId             Datetime  PM2.5    PM10    NO    NO2    NOx    NH3  \
0     AP001  2017-11-24 17:00:00  60.50   98.00  2.35  30.80  18.25   8.50   
1     AP001  2017-11-24 18:00:00  65.50  111.25  2.70  24.20  15.07   9.77   
2     AP001  2017-11-24 19:00:00  80.00  132.00  2.10  25.18  15.15  12.02   
3     AP001  2017-11-24 20:00:00  81.50  133.25  1.95  16.25  10.23  11.58   
4     AP001  2017-11-24 21:00:00  75.25  116.00  1.43  17.48  10.43  12.03   

    CO    SO2      O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  0.1  11.85  126.40      0.1     6.10    0.10  NaN        NaN  
1  0.1  13.17  117.12      0.1     6.25    0.15  NaN        NaN  
2  0.1  12.08   98.98      0.2     5.98    0.18  NaN        NaN  
3  0.1  10.47  112.20      0.2     6.72    0.10  NaN        NaN  
4  0.1   9.12  106.35      0.2     5.75    0.08  NaN        NaN  

Data from /root/.cache/kag

In [ ]:
import pandas as pd
import os

# Assuming the path is correctly set in the previous code execution.
# If not, replace with the actual path.
path = "/root/.cache/kagglehub/datasets/rohanrao/air-quality-data-in-india/versions/12"

stations = pd.read_csv(os.path.join(path, "stations.csv"))
city_hour = pd.read_csv(os.path.join(path, "city_hour.csv"))
station_hour = pd.read_csv(os.path.join(path, "station_hour.csv"))
city_day = pd.read_csv(os.path.join(path, "city_day.csv"))
station_day = pd.read_csv(os.path.join(path, "station_day.csv"))

<ipython-input-114-c0004fb0b5de>:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  station_hour = pd.read_csv(os.path.join(path, "station_hour.csv"))


In [ ]:
# prompt: convert Datetime column to separate Date and Time

# Assuming 'city_hour' DataFrame is already loaded as shown in the previous code.
# Convert the 'Datetime' column to datetime objects if it's not already.
city_hour['Datetime'] = pd.to_datetime(city_hour['Datetime'])

# Extract date and time into separate columns
city_hour['Date'] = city_hour['Datetime'].dt.date
city_hour['Time'] = city_hour['Datetime'].dt.time

# Display the updated DataFrame to verify the changes
print(city_hour.head())

        City            Datetime  PM2.5  PM10    NO    NO2    NOx  NH3    CO  \
0  Ahmedabad 2015-01-01 01:00:00    NaN   NaN  1.00  40.01  36.37  NaN  1.00   
1  Ahmedabad 2015-01-01 02:00:00    NaN   NaN  0.02  27.75  19.73  NaN  0.02   
2  Ahmedabad 2015-01-01 03:00:00    NaN   NaN  0.08  19.32  11.08  NaN  0.08   
3  Ahmedabad 2015-01-01 04:00:00    NaN   NaN  0.30  16.45   9.20  NaN  0.30   
4  Ahmedabad 2015-01-01 05:00:00    NaN   NaN  0.12  14.90   7.85  NaN  0.12   

      SO2      O3  Benzene  Toluene  Xylene  AQI AQI_Bucket        Date  \
0  122.07     NaN      0.0      0.0     0.0  NaN        NaN  2015-01-01   
1   85.90     NaN      0.0      0.0     0.0  NaN        NaN  2015-01-01   
2   52.83     NaN      0.0      0.0     0.0  NaN        NaN  2015-01-01   
3   39.53  153.58      0.0      0.0     0.0  NaN        NaN  2015-01-01   
4   32.63     NaN      0.0      0.0     0.0  NaN        NaN  2015-01-01   

       Time  
0  01:00:00  
1  02:00:00  
2  03:00:00  
3  04:00:00 

In [ ]:
delhi_records = city_hour[city_hour['City'] == 'Delhi']
delhi_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
245172,Delhi,2015-01-01 01:00:00,454.58,935.18,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,NaN,NaN,2015-01-01,01:00:00
245173,Delhi,2015-01-01 02:00:00,440.44,NaN,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,NaN,NaN,2015-01-01,02:00:00
245174,Delhi,2015-01-01 03:00:00,409.09,NaN,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,NaN,NaN,2015-01-01,03:00:00
245175,Delhi,2015-01-01 04:00:00,436.12,NaN,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,NaN,NaN,2015-01-01,04:00:00
245176,Delhi,2015-01-01 05:00:00,415.88,976.99,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,NaN,NaN,2015-01-01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293359,Delhi,2020-06-30 20:00:00,43.19,123.26,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,97.0,Satisfactory,2020-06-30,20:00:00
293360,Delhi,2020-06-30 21:00:00,45.30,125.22,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,99.0,Satisfactory,2020-06-30,21:00:00
293361,Delhi,2020-06-30 22:00:00,49.04,129.39,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,101.0,Moderate,2020-06-30,22:00:00
293362,Delhi,2020-06-30 23:00:00,50.51,128.86,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,102.0,Moderate,2020-06-30,23:00:00


In [ ]:
delhi_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
245172,Delhi,2015-01-01 01:00:00,454.58,935.18,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,NaN,NaN,2015-01-01,01:00:00
245173,Delhi,2015-01-01 02:00:00,440.44,NaN,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,NaN,NaN,2015-01-01,02:00:00
245174,Delhi,2015-01-01 03:00:00,409.09,NaN,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,NaN,NaN,2015-01-01,03:00:00
245175,Delhi,2015-01-01 04:00:00,436.12,NaN,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,NaN,NaN,2015-01-01,04:00:00
245176,Delhi,2015-01-01 05:00:00,415.88,976.99,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,NaN,NaN,2015-01-01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293359,Delhi,2020-06-30 20:00:00,43.19,123.26,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,97.0,Satisfactory,2020-06-30,20:00:00
293360,Delhi,2020-06-30 21:00:00,45.30,125.22,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,99.0,Satisfactory,2020-06-30,21:00:00
293361,Delhi,2020-06-30 22:00:00,49.04,129.39,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,101.0,Moderate,2020-06-30,22:00:00
293362,Delhi,2020-06-30 23:00:00,50.51,128.86,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,102.0,Moderate,2020-06-30,23:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Specify the path to your folder
path_to_folder = "/content/drive/MyDrive/Research_Project_AQI_ExTR_Finalized_Datasets"

# Load the CSV files using their filenames as variables
delhi_records_ema_xtr = pd.read_csv(f"{path_to_folder}/delhi_records_ema_xtr.csv")
delhi_records_knn_impute_xtr = pd.read_csv(f"{path_to_folder}/delhi_records_knn_impute_xtr.csv")
delhi_records_spline_xtr = pd.read_csv(f"{path_to_folder}/delhi_records_spline_xtr.csv")
result_df = pd.read_csv(f"{path_to_folder}/result_df.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
delhi_records = result_df.copy()
delhi_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
0,Delhi,2015-01-01 01:00:00,454.58,935.180000,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,526.280000,NaN,2015-01-01,01:00:00
1,Delhi,2015-01-01 02:00:00,440.44,874.220879,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,524.250000,NaN,2015-01-01,02:00:00
2,Delhi,2015-01-01 03:00:00,409.09,842.413294,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,487.350000,NaN,2015-01-01,03:00:00
3,Delhi,2015-01-01 04:00:00,436.12,881.655247,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,494.516667,NaN,2015-01-01,04:00:00
4,Delhi,2015-01-01 05:00:00,415.88,976.990000,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,477.496667,NaN,2015-01-01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48187,Delhi,2020-06-30 20:00:00,43.19,123.260000,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,97.000000,Satisfactory,2020-06-30,20:00:00
48188,Delhi,2020-06-30 21:00:00,45.30,125.220000,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,99.000000,Satisfactory,2020-06-30,21:00:00
48189,Delhi,2020-06-30 22:00:00,49.04,129.390000,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,101.000000,Moderate,2020-06-30,22:00:00
48190,Delhi,2020-06-30 23:00:00,50.51,128.860000,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,102.000000,Moderate,2020-06-30,23:00:00


# Classification Models

###Practice

In [ ]:
delhi_records.columns

Index(['City', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO',
       'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket',
       'Date', 'Time'],
      dtype='object')

In [ ]:
delhi_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
0,Delhi,2015-01-01 01:00:00,454.58,935.180000,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,526.280000,NaN,2015-01-01,01:00:00
1,Delhi,2015-01-01 02:00:00,440.44,874.220879,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,524.250000,NaN,2015-01-01,02:00:00
2,Delhi,2015-01-01 03:00:00,409.09,842.413294,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,487.350000,NaN,2015-01-01,03:00:00
3,Delhi,2015-01-01 04:00:00,436.12,881.655247,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,494.516667,NaN,2015-01-01,04:00:00
4,Delhi,2015-01-01 05:00:00,415.88,976.990000,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,477.496667,NaN,2015-01-01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48187,Delhi,2020-06-30 20:00:00,43.19,123.260000,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,97.000000,Satisfactory,2020-06-30,20:00:00
48188,Delhi,2020-06-30 21:00:00,45.30,125.220000,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,99.000000,Satisfactory,2020-06-30,21:00:00
48189,Delhi,2020-06-30 22:00:00,49.04,129.390000,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,101.000000,Moderate,2020-06-30,22:00:00
48190,Delhi,2020-06-30 23:00:00,50.51,128.860000,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,102.000000,Moderate,2020-06-30,23:00:00


In [ ]:
# import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# select rows from 'delhi_records' where column='AQI_Bucket' is not NaN
filtered_records = delhi_records[delhi_records['AQI_Bucket'].notna()]
filtered_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
15,Delhi,2015-01-01 16:00:00,211.51,340.66,13.09,37.08,47.07,33.54,15.24,12.33,34.35,3.51,5.85,4.73,456.0,Severe,2015-01-01,16:00:00
16,Delhi,2015-01-01 17:00:00,191.12,257.40,15.44,38.40,51.24,34.98,13.11,18.00,34.38,3.58,5.23,3.78,450.0,Severe,2015-01-01,17:00:00
17,Delhi,2015-01-01 18:00:00,218.44,351.01,22.85,44.73,59.26,34.80,18.35,19.17,38.42,3.67,5.93,4.98,446.0,Severe,2015-01-01,18:00:00
18,Delhi,2015-01-01 19:00:00,296.80,600.95,69.20,47.76,101.68,34.25,16.67,21.50,49.12,14.75,27.13,8.78,475.0,Severe,2015-01-01,19:00:00
19,Delhi,2015-01-01 20:00:00,336.43,714.63,148.42,46.31,171.10,35.75,12.17,17.67,56.44,26.56,45.62,9.99,480.0,Severe,2015-01-01,20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48187,Delhi,2020-06-30 20:00:00,43.19,123.26,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,97.0,Satisfactory,2020-06-30,20:00:00
48188,Delhi,2020-06-30 21:00:00,45.30,125.22,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,99.0,Satisfactory,2020-06-30,21:00:00
48189,Delhi,2020-06-30 22:00:00,49.04,129.39,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,101.0,Moderate,2020-06-30,22:00:00
48190,Delhi,2020-06-30 23:00:00,50.51,128.86,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,102.0,Moderate,2020-06-30,23:00:00


In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'filtered_records' is your dataset and 'feature' and 'target' columns exist

feature = ['AQI']
target = 'AQI_Bucket'

# Check if the target column has any NaN values and filter them out
filtered_records = filtered_records[filtered_records[target].notna()]

# Encode the target column using LabelEncoder
label_encoder = preprocessing.LabelEncoder()
filtered_records[target] = label_encoder.fit_transform(filtered_records[target])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(filtered_records[feature], filtered_records[target], test_size=0.2, random_state=42)

# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        99
           1       1.00      1.00      1.00      2400
           2       1.00      1.00      1.00      2530
           3       1.00      1.00      1.00       862
           4       1.00      1.00      1.00      1181
           5       1.00      1.00      1.00      2467

    accuracy                           1.00      9539
   macro avg       1.00      1.00      1.00      9539
weighted avg       1.00      1.00      1.00      9539

Confusion Matrix:
 [[  99    0    0    0    0    0]
 [   0 2400    0    0    0    0]
 [   0    0 2530    0    0    0]
 [   0    0    0  862    0    0]
 [   0    0    0    0 1181    0]
 [   0    0    0    0    0 2467]]


In [ ]:
NaN_records = delhi_records[delhi_records['AQI_Bucket'].isna()]
NaN_records

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
0,Delhi,2015-01-01 01:00:00,454.58,935.180000,81.52,41.78,187.66,27.540000,9.29,3.410000,54.94,25.24,58.57,13.80,526.280000,NaN,2015-01-01,01:00:00
1,Delhi,2015-01-01 02:00:00,440.44,874.220879,70.80,43.46,176.83,27.720000,13.28,3.880000,50.53,23.10,49.37,15.63,524.250000,NaN,2015-01-01,02:00:00
2,Delhi,2015-01-01 03:00:00,409.09,842.413294,132.46,41.19,141.02,28.940000,29.67,2.830000,19.33,19.04,38.94,17.18,487.350000,NaN,2015-01-01,03:00:00
3,Delhi,2015-01-01 04:00:00,436.12,881.655247,84.78,39.55,102.84,29.300000,21.76,4.330000,20.08,13.99,27.53,16.82,494.516667,NaN,2015-01-01,04:00:00
4,Delhi,2015-01-01 05:00:00,415.88,976.990000,60.24,37.41,80.12,30.840000,26.19,6.170000,16.00,11.14,21.99,14.29,477.496667,NaN,2015-01-01,05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23362,Delhi,2017-08-31 11:00:00,41.29,61.880000,15.13,50.47,29.16,14.804010,0.83,22.427138,22.92,0.27,0.40,0.22,110.603333,NaN,2017-08-31,11:00:00
23363,Delhi,2017-08-31 12:00:00,58.55,83.860000,14.64,51.38,29.19,15.830286,0.79,22.402299,25.76,0.26,0.39,0.22,171.443333,NaN,2017-08-31,12:00:00
23364,Delhi,2017-08-31 13:00:00,63.81,91.300000,14.46,51.05,29.12,14.696563,0.79,23.333788,24.57,0.28,0.37,0.20,185.433333,NaN,2017-08-31,13:00:00
23365,Delhi,2017-08-31 14:00:00,66.11,88.250000,14.28,51.35,29.17,14.273177,0.77,23.591615,25.12,0.23,0.35,0.20,176.436667,NaN,2017-08-31,14:00:00


In [ ]:
X_NaN = NaN_records[['AQI']]
X_NaN

,AQI
0,526.280000
1,524.250000
2,487.350000
3,494.516667
4,477.496667
...,...
23362,110.603333
23363,171.443333
23364,185.433333
23365,176.436667


In [ ]:
# Step 1: Check if there are NaN values in 'AQI_Bucket'
nan_count = delhi_records['AQI_Bucket'].isna().sum()
print(f"Number of missing values in 'AQI_Bucket': {nan_count}")

# Step 2: If there are missing values, proceed with prediction
if nan_count > 0:
    # Isolate the records where AQI_Bucket is NaN
    NaN_records = delhi_records[delhi_records['AQI_Bucket'].isna()]

    # Step 3: Scale the 'AQI' values for NaN records using the same scaler from training
    X_NaN_scaled = scaler.transform(NaN_records[['AQI']])

    # Step 4: Predict the 'AQI_Bucket' values using the trained Random Forest model
    predicted_AQI_Bucket = model_rf.predict(X_NaN_scaled)

    # Step 5: Convert numeric predictions back to original text labels
    predicted_AQI_Bucket_text = label_encoder.inverse_transform(predicted_AQI_Bucket)

    # Step 6: Update the 'AQI_Bucket' column in NaN_records with the predicted text labels
    NaN_records['AQI_Bucket'] = predicted_AQI_Bucket_text

    # Step 7: Optionally, print the updated NaN_records to see the results
    print(NaN_records)

    # Step 8: Combine the original data (without NaN values) with the updated NaN_records
    rfc_dataset = pd.concat([delhi_records.dropna(subset=['AQI_Bucket']), NaN_records])
    print(rfc_dataset)

else:
    print("No missing values in 'AQI_Bucket'.")


Number of missing values in 'AQI_Bucket': 498
        City             Datetime   PM2.5        PM10      NO    NO2     NOx  \
0      Delhi  2015-01-01 01:00:00  454.58  935.180000   81.52  41.78  187.66   
1      Delhi  2015-01-01 02:00:00  440.44  874.220879   70.80  43.46  176.83   
2      Delhi  2015-01-01 03:00:00  409.09  842.413294  132.46  41.19  141.02   
3      Delhi  2015-01-01 04:00:00  436.12  881.655247   84.78  39.55  102.84   
4      Delhi  2015-01-01 05:00:00  415.88  976.990000   60.24  37.41   80.12   
...      ...                  ...     ...         ...     ...    ...     ...   
23362  Delhi  2017-08-31 11:00:00   41.29   61.880000   15.13  50.47   29.16   
23363  Delhi  2017-08-31 12:00:00   58.55   83.860000   14.64  51.38   29.19   
23364  Delhi  2017-08-31 13:00:00   63.81   91.300000   14.46  51.05   29.12   
23365  Delhi  2017-08-31 14:00:00   66.11   88.250000   14.28  51.35   29.17   
23366  Delhi  2017-08-31 15:00:00   46.12   57.180000   13.49  54.24   30.

<ipython-input-126-c07baeeddc11>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NaN_records['AQI_Bucket'] = predicted_AQI_Bucket_text


In [ ]:
rfc_dataset

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
15,Delhi,2015-01-01 16:00:00,211.51,340.66,13.09,37.08,47.07,33.540000,15.24,12.330000,34.35,3.51,5.85,4.73,456.000000,Severe,2015-01-01,16:00:00
16,Delhi,2015-01-01 17:00:00,191.12,257.40,15.44,38.40,51.24,34.980000,13.11,18.000000,34.38,3.58,5.23,3.78,450.000000,Severe,2015-01-01,17:00:00
17,Delhi,2015-01-01 18:00:00,218.44,351.01,22.85,44.73,59.26,34.800000,18.35,19.170000,38.42,3.67,5.93,4.98,446.000000,Severe,2015-01-01,18:00:00
18,Delhi,2015-01-01 19:00:00,296.80,600.95,69.20,47.76,101.68,34.250000,16.67,21.500000,49.12,14.75,27.13,8.78,475.000000,Severe,2015-01-01,19:00:00
19,Delhi,2015-01-01 20:00:00,336.43,714.63,148.42,46.31,171.10,35.750000,12.17,17.670000,56.44,26.56,45.62,9.99,480.000000,Severe,2015-01-01,20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23362,Delhi,2017-08-31 11:00:00,41.29,61.88,15.13,50.47,29.16,14.804010,0.83,22.427138,22.92,0.27,0.40,0.22,110.603333,Moderate,2017-08-31,11:00:00
23363,Delhi,2017-08-31 12:00:00,58.55,83.86,14.64,51.38,29.19,15.830286,0.79,22.402299,25.76,0.26,0.39,0.22,171.443333,Moderate,2017-08-31,12:00:00
23364,Delhi,2017-08-31 13:00:00,63.81,91.30,14.46,51.05,29.12,14.696563,0.79,23.333788,24.57,0.28,0.37,0.20,185.433333,Moderate,2017-08-31,13:00:00
23365,Delhi,2017-08-31 14:00:00,66.11,88.25,14.28,51.35,29.17,14.273177,0.77,23.591615,25.12,0.23,0.35,0.20,176.436667,Moderate,2017-08-31,14:00:00


## Random Forest Classifier

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'delhi_records' is your main dataset, and 'feature' and 'target' columns exist

feature_rf = ['AQI']
target_rf = 'AQI_Bucket'

# Filter out rows where 'AQI_Bucket' is NaN for training
filtered_records_rf = delhi_records[delhi_records[target_rf].notna()]

# Encode the target column using LabelEncoder
label_encoder_rf = preprocessing.LabelEncoder()
filtered_records_rf[target_rf] = label_encoder_rf.fit_transform(filtered_records_rf[target_rf])

# Split the data into training and testing sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(filtered_records_rf[feature_rf], filtered_records_rf[target_rf], test_size=0.2, random_state=42)

# Standardize the features using StandardScaler
scaler_rf = StandardScaler()
X_train_rf = scaler_rf.fit_transform(X_train_rf)
X_test_rf = scaler_rf.transform(X_test_rf)

# Train a RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_rf, y_train_rf)

# Make predictions and evaluate the model
y_pred_rf = model_rf.predict(X_test_rf)
print("Classification Report:\n", classification_report(y_test_rf, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test_rf, y_pred_rf))

# Check if there are NaN values in 'AQI_Bucket' of delhi_records
nan_count_rf = delhi_records['AQI_Bucket'].isna().sum()
print(f"Number of missing values in 'AQI_Bucket': {nan_count_rf}")

# Step 2: If there are missing values, proceed with prediction
if nan_count_rf > 0:
    # Isolate the records where AQI_Bucket is NaN
    NaN_records_rf = delhi_records[delhi_records['AQI_Bucket'].isna()]

    # Step 3: Scale the 'AQI' values for NaN records using the same scaler from training
    X_NaN_scaled_rf = scaler_rf.transform(NaN_records_rf[['AQI']])

    # Step 4: Predict the 'AQI_Bucket' values using the trained Random Forest model
    predicted_AQI_Bucket_rf = model_rf.predict(X_NaN_scaled_rf)

    # Step 5: Convert numeric predictions back to original text labels
    predicted_AQI_Bucket_text_rf = label_encoder_rf.inverse_transform(predicted_AQI_Bucket_rf)

    # Step 6: Update the 'AQI_Bucket' column in NaN_records with the predicted text labels
    NaN_records_rf['AQI_Bucket'] = predicted_AQI_Bucket_text_rf

    # Step 7: Optionally, print the updated NaN_records to see the results
    print(NaN_records_rf)

    # Step 8: Combine the original data (without NaN values) with the updated NaN_records
    dataset_rf = pd.concat([delhi_records.dropna(subset=['AQI_Bucket']), NaN_records_rf])
    print(dataset_rf)

else:
    print("No missing values in 'AQI_Bucket'.")

dataset_rf

<ipython-input-128-62f759fee5a5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_records_rf[target_rf] = label_encoder_rf.fit_transform(filtered_records_rf[target_rf])


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        99
           1       1.00      1.00      1.00      2400
           2       1.00      1.00      1.00      2530
           3       1.00      1.00      1.00       862
           4       1.00      1.00      1.00      1181
           5       1.00      1.00      1.00      2467

    accuracy                           1.00      9539
   macro avg       1.00      1.00      1.00      9539
weighted avg       1.00      1.00      1.00      9539

Confusion Matrix:
 [[  99    0    0    0    0    0]
 [   0 2400    0    0    0    0]
 [   0    0 2530    0    0    0]
 [   0    0    0  862    0    0]
 [   0    0    0    0 1181    0]
 [   0    0    0    0    0 2467]]
Number of missing values in 'AQI_Bucket': 498
        City             Datetime   PM2.5        PM10      NO    NO2     NOx  \
0      Delhi  2015-01-01 01:00:00  454.58  935.180000   81.52  41.78  187.66   
1      De

<ipython-input-128-62f759fee5a5>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NaN_records_rf['AQI_Bucket'] = predicted_AQI_Bucket_text_rf


,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
15,Delhi,2015-01-01 16:00:00,211.51,340.66,13.09,37.08,47.07,33.540000,15.24,12.330000,34.35,3.51,5.85,4.73,456.000000,Severe,2015-01-01,16:00:00
16,Delhi,2015-01-01 17:00:00,191.12,257.40,15.44,38.40,51.24,34.980000,13.11,18.000000,34.38,3.58,5.23,3.78,450.000000,Severe,2015-01-01,17:00:00
17,Delhi,2015-01-01 18:00:00,218.44,351.01,22.85,44.73,59.26,34.800000,18.35,19.170000,38.42,3.67,5.93,4.98,446.000000,Severe,2015-01-01,18:00:00
18,Delhi,2015-01-01 19:00:00,296.80,600.95,69.20,47.76,101.68,34.250000,16.67,21.500000,49.12,14.75,27.13,8.78,475.000000,Severe,2015-01-01,19:00:00
19,Delhi,2015-01-01 20:00:00,336.43,714.63,148.42,46.31,171.10,35.750000,12.17,17.670000,56.44,26.56,45.62,9.99,480.000000,Severe,2015-01-01,20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23362,Delhi,2017-08-31 11:00:00,41.29,61.88,15.13,50.47,29.16,14.804010,0.83,22.427138,22.92,0.27,0.40,0.22,110.603333,Moderate,2017-08-31,11:00:00
23363,Delhi,2017-08-31 12:00:00,58.55,83.86,14.64,51.38,29.19,15.830286,0.79,22.402299,25.76,0.26,0.39,0.22,171.443333,Moderate,2017-08-31,12:00:00
23364,Delhi,2017-08-31 13:00:00,63.81,91.30,14.46,51.05,29.12,14.696563,0.79,23.333788,24.57,0.28,0.37,0.20,185.433333,Moderate,2017-08-31,13:00:00
23365,Delhi,2017-08-31 14:00:00,66.11,88.25,14.28,51.35,29.17,14.273177,0.77,23.591615,25.12,0.23,0.35,0.20,176.436667,Moderate,2017-08-31,14:00:00


In [ ]:
# check if dataset has any other NaN values
dataset_rf.isna().sum()

,0
City,0
Datetime,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0
SO2,0


In [ ]:
# save the csv file
dataset_rf.to_csv('completed_dataset_rfc.csv', index=False)

## Decision Tree Classification

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier  # Changed from RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'delhi_records' is your main dataset, and 'feature' and 'target' columns exist

feature_dt = ['AQI']
target_dt = 'AQI_Bucket'

# Filter out rows where 'AQI_Bucket' is NaN for training
filtered_records_dt = delhi_records[delhi_records[target_dt].notna()]

# Encode the target column using LabelEncoder
label_encoder_dt = preprocessing.LabelEncoder()
filtered_records_dt[target_dt] = label_encoder_dt.fit_transform(filtered_records_dt[target_dt])

# Split the data into training and testing sets
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(filtered_records_dt[feature_dt], filtered_records_dt[target_dt], test_size=0.2, random_state=42)

# Standardize the features using StandardScaler
scaler_dt = StandardScaler()
X_train_dt = scaler_dt.fit_transform(X_train_dt)
X_test_dt = scaler_dt.transform(X_test_dt)

# Train a DecisionTreeClassifier instead of RandomForest
model_dt = DecisionTreeClassifier(random_state=42)  # Changed to DecisionTreeClassifier
model_dt.fit(X_train_dt, y_train_dt)

# Make predictions and evaluate the model
y_pred_dt = model_dt.predict(X_test_dt)
print("Classification Report:\n", classification_report(y_test_dt, y_pred_dt))
print("Confusion Matrix:\n", confusion_matrix(y_test_dt, y_pred_dt))

# Check if there are NaN values in 'AQI_Bucket' of delhi_records
nan_count_dt = delhi_records['AQI_Bucket'].isna().sum()
print(f"Number of missing values in 'AQI_Bucket': {nan_count_dt}")

# Step 2: If there are missing values, proceed with prediction
if nan_count_dt > 0:
    # Isolate the records where AQI_Bucket is NaN
    NaN_records_dt = delhi_records[delhi_records['AQI_Bucket'].isna()]

    # Step 3: Scale the 'AQI' values for NaN records using the same scaler from training
    X_NaN_scaled_dt = scaler_dt.transform(NaN_records_dt[['AQI']])

    # Step 4: Predict the 'AQI_Bucket' values using the trained Decision Tree model
    predicted_AQI_Bucket_dt = model_dt.predict(X_NaN_scaled_dt)

    # Step 5: Convert numeric predictions back to original text labels
    predicted_AQI_Bucket_text_dt = label_encoder_dt.inverse_transform(predicted_AQI_Bucket_dt)

    # Step 6: Update the 'AQI_Bucket' column in NaN_records with the predicted text labels
    NaN_records_dt['AQI_Bucket'] = predicted_AQI_Bucket_text_dt

    # Step 7: Optionally, print the updated NaN_records to see the results
    print(NaN_records_dt)

    # Step 8: Combine the original data (without NaN values) with the updated NaN_records
    dt_dataset = pd.concat([delhi_records.dropna(subset=['AQI_Bucket']), NaN_records_dt])
    print(dt_dataset)

else:
    print("No missing values in 'AQI_Bucket'.")

dt_dataset

<ipython-input-131-c1d20e560e12>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_records_dt[target_dt] = label_encoder_dt.fit_transform(filtered_records_dt[target_dt])
<ipython-input-131-c1d20e560e12>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NaN_records_dt['AQI_Bucket'] = predicted_AQI_Bucket_text_dt


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        99
           1       1.00      1.00      1.00      2400
           2       1.00      1.00      1.00      2530
           3       1.00      1.00      1.00       862
           4       1.00      1.00      1.00      1181
           5       1.00      1.00      1.00      2467

    accuracy                           1.00      9539
   macro avg       1.00      1.00      1.00      9539
weighted avg       1.00      1.00      1.00      9539

Confusion Matrix:
 [[  99    0    0    0    0    0]
 [   0 2400    0    0    0    0]
 [   0    0 2530    0    0    0]
 [   0    0    0  862    0    0]
 [   0    0    0    0 1181    0]
 [   0    0    0    0    0 2467]]
Number of missing values in 'AQI_Bucket': 498
        City             Datetime   PM2.5        PM10      NO    NO2     NOx  \
0      Delhi  2015-01-01 01:00:00  454.58  935.180000   81.52  41.78  187.66   
1      De

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Date,Time
15,Delhi,2015-01-01 16:00:00,211.51,340.66,13.09,37.08,47.07,33.540000,15.24,12.330000,34.35,3.51,5.85,4.73,456.000000,Severe,2015-01-01,16:00:00
16,Delhi,2015-01-01 17:00:00,191.12,257.40,15.44,38.40,51.24,34.980000,13.11,18.000000,34.38,3.58,5.23,3.78,450.000000,Severe,2015-01-01,17:00:00
17,Delhi,2015-01-01 18:00:00,218.44,351.01,22.85,44.73,59.26,34.800000,18.35,19.170000,38.42,3.67,5.93,4.98,446.000000,Severe,2015-01-01,18:00:00
18,Delhi,2015-01-01 19:00:00,296.80,600.95,69.20,47.76,101.68,34.250000,16.67,21.500000,49.12,14.75,27.13,8.78,475.000000,Severe,2015-01-01,19:00:00
19,Delhi,2015-01-01 20:00:00,336.43,714.63,148.42,46.31,171.10,35.750000,12.17,17.670000,56.44,26.56,45.62,9.99,480.000000,Severe,2015-01-01,20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23362,Delhi,2017-08-31 11:00:00,41.29,61.88,15.13,50.47,29.16,14.804010,0.83,22.427138,22.92,0.27,0.40,0.22,110.603333,Moderate,2017-08-31,11:00:00
23363,Delhi,2017-08-31 12:00:00,58.55,83.86,14.64,51.38,29.19,15.830286,0.79,22.402299,25.76,0.26,0.39,0.22,171.443333,Moderate,2017-08-31,12:00:00
23364,Delhi,2017-08-31 13:00:00,63.81,91.30,14.46,51.05,29.12,14.696563,0.79,23.333788,24.57,0.28,0.37,0.20,185.433333,Moderate,2017-08-31,13:00:00
23365,Delhi,2017-08-31 14:00:00,66.11,88.25,14.28,51.35,29.17,14.273177,0.77,23.591615,25.12,0.23,0.35,0.20,176.436667,Moderate,2017-08-31,14:00:00


In [ ]:
# save the dataset
dt_dataset.to_csv('completed_dataset_dtc.csv', index=False)